In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
import os
from PIL import Image
import random
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import math as Math
from keras.applications.densenet import DenseNet121

fpath = "../input/stanford/stanford-dogs-20-breeds/images_trn_BIG"
#fpath = "./Dogs/Images/Images5"


CATEGORIES_AMOUNT = len(os.listdir(fpath))
#AMOUNT_PER_CATEGORY_TRAIN = 20
#AMOUNT_PER_CATEGORY_TEST = 10


categories = os.listdir(fpath)
categories = categories[:CATEGORIES_AMOUNT]

img_width = 224
img_height = 224
batch_size = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/stanford/stanford-dogs-20-breeds/images_trn_BIG",
    validation_split=0.2,
    subset='training',
    seed=42069,
    image_size=(img_width, img_height),
    batch_size=batch_size,
    label_mode="categorical"
)

class_names = train_ds.class_names
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/stanford/stanford-dogs-20-breeds/images_trn_BIG",
    validation_split=0.2,
    subset='validation',
    seed=42069,
    image_size=(img_width, img_height),
    batch_size=batch_size,
    label_mode="categorical"
)


#TrainData = tf.keras.preprocessing.image.ImageDataGenerator(
#    width_shift_range=0.2, height_shift_range=0.2,
#    rotation_range=90, rescale=1/255,
#    horizontal_flip=True, vertical_flip=True)

#ValidData = tf.keras.preprocessing.image.ImageDataGenerator()


#train_gen = TrainData.flow_from_directory(
#    directory="../Dogs/Images/Images_trn",
#    target_size=(img_width, img_height),
#    batch_size=batch_size)
#
#val_gen = ValidData.flow_from_directory(
#    directory="./Dogs/Images/Images_val",
#    target_size=(img_width, img_height),
#    batch_size=batch_size)


#img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#    #width_shift_range=0.2, height_shift_range=0.2,
#    #rotation_range=90, 
#    rescale=1/255,
#    horizontal_flip=True, vertical_flip=True).flow_from_directory(
#    directory="../input/stanford-dogs-dataset/images/Images",
#    target_size=(img_width, img_height),
#    batch_size=batch_size)

#for image_batch, labels_batch in train_ds:
#    print(image_batch.shape)
#    print(labels_batch.shape)
#    break


#normalization_layer = tf.keras.layers.Rescaling(1./255)
#normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
#image_batch, labels_batch = next(iter(normalized_ds))
#first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
#print(np.min(first_image), np.max(first_image))


AUTOTUNE = tf.data.AUTOTUNE

Found 3043 files belonging to 18 classes.
Using 2435 files for training.


2022-06-02 18:12:16.627258: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Found 3043 files belonging to 18 classes.
Using 608 files for validation.


In [2]:
# # # На основе заранее тренированной сети (224,224,3)
inputl1 = tf.keras.Input((224,224,3))
backbone = DenseNet121(input_shape=(224,224,3),
                       input_tensor = inputl1,
                       weights="../input/densenet-keras/DenseNet-BC-121-32-no-top.h5",
                       include_top=False)
modell = backbone.output
modell = tf.keras.layers.GlobalAveragePooling2D()(modell)
modell = tf.keras.layers.Dense(1024, activation="relu")(modell)
modell = tf.keras.layers.Dropout(0.5)(modell)
modell = tf.keras.layers.Dense(512, activation="relu")(modell)
modell = tf.keras.layers.Dropout(0.5)(modell)
outpl1 = tf.keras.layers.Dense(CATEGORIES_AMOUNT, activation="softmax")(modell)
model = tf.keras.Model(inputl1, outpl1)
for layer in model.layers[:-6]:
    layer.trainable = False

In [3]:
# # # 
# model = tf.keras.Sequential([
#   tf.keras.layers.Rescaling(1./255),
#   tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
#   tf.keras.layers.Conv2D(32, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Dropout(0.25),
#   tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
#   tf.keras.layers.Conv2D(64, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Dropout(0.25),
#   tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
#   tf.keras.layers.Conv2D(64, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Dropout(0.25),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(512, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
#   tf.keras.layers.Dense(CATEGORIES_AMOUNT, activation='softmax')
# ])

In [4]:
# # # 
#model = tf.keras.Sequential([
#    tf.keras.layers.Rescaling(1./255),
#    tf.keras.layers.Conv2D(16, 3,  activation='relu'),
#    tf.keras.layers.Conv2D(16, 3, activation='relu'),
#    tf.keras.layers.MaxPooling2D(),
#    tf.keras.layers.Conv2D(32, 3, activation='relu'),
#    tf.keras.layers.Conv2D(32, 3,  activation='relu'),
#    tf.keras.layers.MaxPooling2D(),
#    tf.keras.layers.Conv2D(64, 3,  activation='relu'),
#    tf.keras.layers.MaxPooling2D(),
#    tf.keras.layers.Dropout(0.2),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(128, activation='relu'),
#    tf.keras.layers.Dense(CATEGORIES_AMOUNT)
#])


In [5]:
## # Из keggle примера (227,227,3)
#model = tf.keras.Sequential([
#    tf.keras.layers.Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding="valid",activation="relu",input_shape=(227,227,3)),
#    tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding="valid",activation="relu"),
#    tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"),
#    tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"),
#    tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"),
#    tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(4096,input_shape=(227,227,3),activation="relu"),
#    tf.keras.layers.Dropout(0.4),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Dense(4096,activation="relu"),
#    tf.keras.layers.Dropout(0.4),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Dense(1000,activation="relu"),
#    tf.keras.layers.Dropout(0.4),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Dense(23,activation="softmax")
#    ])

In [6]:

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 5

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience = 10,
                                                  mode='min')

In [7]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    #validation_steps=len(val_gen),
                    #steps_per_epoch=len(train_gen),
                    callbacks=[early_stopping],
                    epochs=epochs)
#model.summary() #если это DenseNet, то лучше закомментировать т.к. она большая и её структура известна.

2022-06-02 18:12:26.317580: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


153/153 [==============================] - 190s 1s/step - loss: 4.2080 - accuracy: 0.0719 - val_loss: 2.8525 - val_accuracy: 0.0905


In [8]:

train_loss = history.history['loss']
train_acc = history.history['accuracy']
val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']

epochs_range = range(epochs)

predictions = model.predict(val_ds)


def plot_value_array(predictions_array, true_labels_array):
    #print(predictions_array, true_labels_array)
    plt.grid(False)
    plt.xticks(range(CATEGORIES_AMOUNT))
    plt.yticks([])
    thisplot = plt.bar(range(CATEGORIES_AMOUNT),
                       predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)
    true_label = np.argmax(true_labels_array)
    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')


def plot_image(predictions_array, true_labels_array, img):
    #print(predictions_array, true_labels_array, img)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img.numpy().astype("uint8"))
    predicted_label = np.argmax(predictions_array)
    true_label = np.argmax(true_labels_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                         100*np.max(predictions_array),
                                         class_names[true_label]),
               color=color)


In [9]:
print(history)

In [10]:
epochs = 5
#print("Рисовать графики? [0|1]: ", end='  ')
#j = int(input())
#if(j == 1):
if(True):
    plt.figure(figsize=(12, 10))
    plt.plot(range(epochs), train_loss, label="Training Loss")
    plt.plot(range(epochs), val_loss, label="Validation Loss")
    plt.legend(loc='upper left')
    plt.title('Training and Validation Loss')
    plt.show()

    #plt.figure(figsize=(12, 52))
    #plt.plot(range(epochs), train_acc, label="Training Accuracy")
    #plt.plot(range(epochs), val_acc, label="Validation Accuracy")
    #plt.legend(loc='upper left')
    #plt.title('Training and Validation Accuracy')
    #plt.show()

Рисовать графики? [0|1]:   

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
#print("Показывать по 1 картинки на категорию? [0|1]: ", end='  ')
#j = int(input())
#if(j == 1):
if(True):
    index = 1
    good = []
    for images, labels in val_ds.take(1):
        for i in range(CATEGORIES_AMOUNT):
            good.append(i)

    plt.figure(figsize=(int(Math.sqrt(len(good))) +
               10, 2*(int(Math.sqrt(len(good)))+10)))
    for images, labels in val_ds.take(1):
        for i in good:
            ax = plt.subplot(int(Math.sqrt(len(good))) + 1,
                             2*(int(Math.sqrt(len(good)))+1), index)
            index += 1
            # plt.imshow(images[i].numpy().astype("uint8"))
            plot_image(predictions[i], labels[i], images[i])
            plt.subplot(int(Math.sqrt(len(good))) + 1, 2 *
                        (int(Math.sqrt(len(good)))+1), index)
            index += 1
            plot_value_array(predictions[i], labels[i])
            plt.axis("off")
            print(str(index//2)+"/"+str(len(good)))

    plt.show()

In [ ]:

#print("Показывать все угаданные? [0|1]: ", end='  ')
#j = int(input())
#if(j == 1):
if(True):
    index = 1
    good = []
    for images, labels in val_ds.take(100):
        for i in range(CATEGORIES_AMOUNT):
            if((np.argmax(predictions[i]) == np.argmax(labels))):
                #print(predictions[i], labels)
                good.append(i)

    plt.figure(figsize=(int(Math.sqrt(len(good))) +
               1, 2*(int(Math.sqrt(len(good)))+1)))
    for images, labels in val_ds.take(1):
        for i in good:
            ax = plt.subplot(int(Math.sqrt(len(good))) + 1,
                             2*(int(Math.sqrt(len(good)))+1), index)
            index += 1
            # plt.imshow(images[i].numpy().astype("uint8"))
            print(predictions.shape, labels.shape, images.shape)
            plot_image(predictions[i], labels[i][1], images[i])
            plt.subplot(int(Math.sqrt(len(good))) + 1, 2 *
                        (int(Math.sqrt(len(good)))+1), index)
            index += 1
            plot_value_array(predictions[i], i)
            plt.axis("off")
            print(str(index//2)+"/"+str(len(good)))

    plt.show()

In [ ]:


print("Введите номер картинки для проверки: ", end='  ')
j = int(input())
if(j != 0):
    good = []
    index = 1
    z = 0
    for images, labels in val_ds.take(100):
        for i in range(CATEGORIES_AMOUNT):
            z += 1
            if(z == j):
                good.append(i)

    plt.figure(figsize=(int(10*Math.sqrt(len(good))) +
               10, 2*(int(Math.sqrt(len(good)))+10)))
    for images, labels in val_ds.take(1):
        for i in good:
            ax = plt.subplot(int(Math.sqrt(len(good))) + 1,
                             2*(int(Math.sqrt(len(good)))+1), index)
            index += 1
            # plt.imshow(images[i].numpy().astype("uint8"))
            plot_image(predictions[i], labels[i], images[i])
            plt.subplot(int(Math.sqrt(len(good))) + 1, 2 *
                        (int(Math.sqrt(len(good)))+1), index)
            index += 1
            plot_value_array(predictions[i], i)
            plt.axis("off")
            print(str(index//2)+"/"+str(len(good)))
    plt.show()
